Python не имеет строгой **инкапсуляции** (*encapsulation*) как Java или C++. Вместо этого используются соглашения об именовании и механизм **искажения имён** (*name mangling*).

Уровни доступа в Python (Access Modifiers)

| Синтаксис | Название | Поведение |
|-----------|----------|-----------|
| `name` | публичный (public) | Обычный доступ |
| `_name` | внутренний (internal) | Соглашение: «не трогать» |
| `__name` | искажённый (mangled) | Переименовывается в `_Class__name` |

Механизм искажения имен работает следующим образом. Если используется атрибут с именем `__name`, то при определении класса `Class` применяется искажение имени `__name`, и имя везде заменяется на `_Class__name`. В итоге, аритбута с именем `__name` не существуют. Все методы работают с именем `_Class__name`. Искажение имен в перую очередь предназначен для предотвращение конфликтов имён в подклассах.

>Искажение имён не применяется к именам вида __name__ (с двумя подчёркиваниями с обеих сторон). Это специальные атрибуты, и они остаются без изменений.

In [30]:
class Cell:
    """Биологическая клетка."""
    
    def __init__(self, cell_type, diameter_um):
        self.cell_type = cell_type        # публичный: тип клетки
        self._organelles = []             # внутренний: список органелл
        self.__atp_level = 100.0          # "приватный": уровень АТФ
        self._diameter = diameter_um
    
    def add_organelle(self, name):
        """Добавить органеллу (organelle)."""
        self._organelles.append(name)
    
    def _produce_atp(self, amount):
        """Внутренний метод: синтез АТФ в митохондриях."""
        self.__atp_level += amount
    
    def __regulate_metabolism(self):
        """'Приватный' метод: регуляция метаболизма."""
        if self.__atp_level < 20:
            self._produce_atp(50)
    
    def get_status(self):
        """Публичный интерфейс для получения состояния."""
        self.__regulate_metabolism()
        return {
            "type": self.cell_type,
            "atp": self.__atp_level,
            "organelles": self._organelles
        }

In [31]:
neuron = Cell("neuron", 15.0)
neuron.add_organelle("nucleus")
neuron.add_organelle("mitochondria")

In [32]:
# neuron.__atp_level    # нет доступа к полю __atp_level

In [33]:
neuron.__dict__     # содержит искаженное имя для атрибута __atp_level

{'cell_type': 'neuron',
 '_organelles': ['nucleus', 'mitochondria'],
 '_Cell__atp_level': 100.0,
 '_diameter': 15.0}

Несмотря на то, что мы не можем получить доступ к приватному полю `__atp_level` через имя `__atp_level`, мы тем не менн можем получить доступ к нему через искаженное имя:

In [38]:
neuron._Cell__atp_level     # доступ к приватному атрибуту __atp_level 
# neuron._Cell__atp_level = 300     # модификация приватного атрибута

100.0

Если попробовать присвоить приватному аргументу значение (через имя `__atp_level`), то к объекту динамически добавится поле с именем `__atp_level`. 

In [40]:
neuron.__atp_level = 200

In [41]:
neuron.__dict__

{'cell_type': 'neuron',
 '_organelles': ['nucleus', 'mitochondria'],
 '_Cell__atp_level': 100.0,
 '_diameter': 15.0,
 '__atp_level': 200}

И теперь чтение поля `__atp_level` не приводит к ошибке:

In [42]:
neuron.__atp_level

200

Методы объекта продолжают работать с искаженным именем `_Cell__atp_level`, т.е. с оригинальным полем, поэтому конфликтов не возникает.

In [45]:
neuron.get_status()["atp"]

100.0